Project Overview
The Movie Database Analysis project is all about creating a relational database that stores data about movies, directors, genres, actors, and their relationships. Once we set up the database, we can use SQL queries to derive interesting insights, such as identifying top-rated movies, popular actors, frequent collaborations, and more.

This project gives us experience in the following areas:

Database Design: Creating tables and defining relationships.
Data Insertion: Adding data to the tables to simulate a real-world database.
SQL Querying: Writing and running SQL queries to extract meaningful insights.
Complex Queries: Running complex queries to uncover hidden relationships between data points.

In [ ]:
import sqlite3
import pandas as pd

In [7]:
# Connect to an SQLite database (or create one if it doesn't exist)
connection = sqlite3.connect('movie_database.db')
cursor = connection.cursor()


In [8]:
# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS Movies (
    movie_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    release_year INTEGER,
    director_id INTEGER,
    duration_minutes INTEGER
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Directors (
    director_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    birth_year INTEGER
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Genres (
    genre_id INTEGER PRIMARY KEY,
    genre_name TEXT NOT NULL
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Movie_Genres (
    movie_id INTEGER,
    genre_id INTEGER,
    PRIMARY KEY (movie_id, genre_id),
    FOREIGN KEY (movie_id) REFERENCES Movies(movie_id),
    FOREIGN KEY (genre_id) REFERENCES Genres(genre_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Ratings (
    rating_id INTEGER PRIMARY KEY,
    movie_id INTEGER,
    user_rating DECIMAL(2, 1),
    FOREIGN KEY (movie_id) REFERENCES Movies(movie_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Actors (
    actor_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    birth_year INTEGER
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Movie_Actors (
    movie_id INTEGER,
    actor_id INTEGER,
    PRIMARY KEY (movie_id, actor_id),
    FOREIGN KEY (movie_id) REFERENCES Movies(movie_id),
    FOREIGN KEY (actor_id) REFERENCES Actors(actor_id)
);
''')

# Insert sample data into tables
# Directors
cursor.execute("INSERT OR IGNORE INTO Directors VALUES (1, 'Christopher Nolan', 1970);")
cursor.execute("INSERT OR IGNORE INTO Directors VALUES (2, 'Quentin Tarantino', 1963);")
cursor.execute("INSERT OR IGNORE INTO Directors VALUES (3, 'Steven Spielberg', 1946);")

# Movies
cursor.execute("INSERT OR IGNORE INTO Movies VALUES (1, 'Inception', 2010, 1, 148);")
cursor.execute("INSERT OR IGNORE INTO Movies VALUES (2, 'Pulp Fiction', 1994, 2, 154);")
cursor.execute("INSERT OR IGNORE INTO Movies VALUES (3, 'Jurassic Park', 1993, 3, 127);")
cursor.execute("INSERT OR IGNORE INTO Movies VALUES (4, 'Django Unchained', 2012, 2, 165);")

# Genres
cursor.execute("INSERT OR IGNORE INTO Genres VALUES (1, 'Action');")
cursor.execute("INSERT OR IGNORE INTO Genres VALUES (2, 'Thriller');")
cursor.execute("INSERT OR IGNORE INTO Genres VALUES (3, 'Sci-Fi');")
cursor.execute("INSERT OR IGNORE INTO Genres VALUES (4, 'Adventure');")
cursor.execute("INSERT OR IGNORE INTO Genres VALUES (5, 'Drama');")

# Movie_Genres
cursor.execute("INSERT OR IGNORE INTO Movie_Genres VALUES (1, 1);")
cursor.execute("INSERT OR IGNORE INTO Movie_Genres VALUES (1, 2);")
cursor.execute("INSERT OR IGNORE INTO Movie_Genres VALUES (1, 3);")
cursor.execute("INSERT OR IGNORE INTO Movie_Genres VALUES (2, 5);")
cursor.execute("INSERT OR IGNORE INTO Movie_Genres VALUES (3, 4);")
cursor.execute("INSERT OR IGNORE INTO Movie_Genres VALUES (3, 3);")
cursor.execute("INSERT OR IGNORE INTO Movie_Genres VALUES (4, 1);")
cursor.execute("INSERT OR IGNORE INTO Movie_Genres VALUES (4, 5);")

# Ratings
cursor.execute("INSERT OR IGNORE INTO Ratings VALUES (1, 1, 8.8);")
cursor.execute("INSERT OR IGNORE INTO Ratings VALUES (2, 2, 8.9);")
cursor.execute("INSERT OR IGNORE INTO Ratings VALUES (3, 3, 8.1);")
cursor.execute("INSERT OR IGNORE INTO Ratings VALUES (4, 4, 8.4);")

# Actors
cursor.execute("INSERT OR IGNORE INTO Actors VALUES (1, 'Leonardo DiCaprio', 1974);")
cursor.execute("INSERT OR IGNORE INTO Actors VALUES (2, 'Samuel L. Jackson', 1948);")
cursor.execute("INSERT OR IGNORE INTO Actors VALUES (3, 'Laura Dern', 1967);")
cursor.execute("INSERT OR IGNORE INTO Actors VALUES (4, 'Jamie Foxx', 1967);")

# Movie_Actors
cursor.execute("INSERT OR IGNORE INTO Movie_Actors VALUES (1, 1);")  # Inception: Leonardo DiCaprio
cursor.execute("INSERT OR IGNORE INTO Movie_Actors VALUES (2, 2);")  # Pulp Fiction: Samuel L. Jackson
cursor.execute("INSERT OR IGNORE INTO Movie_Actors VALUES (3, 3);")  # Jurassic Park: Laura Dern
cursor.execute("INSERT OR IGNORE INTO Movie_Actors VALUES (4, 2);")  # Django Unchained: Samuel L. Jackson
cursor.execute("INSERT OR IGNORE INTO Movie_Actors VALUES (4, 4);")  # Django Unchained: Jamie Foxx

# Commit the changes
connection.commit()


In [9]:
# Find the top-rated movies
query = '''
SELECT title, user_rating
FROM Movies
JOIN Ratings ON Movies.movie_id = Ratings.movie_id
ORDER BY user_rating DESC
LIMIT 5;
'''

# Execute the query and display results in a DataFrame
result = pd.read_sql_query(query, connection)
result


,title,user_rating
0,Pulp Fiction,8.9
1,Inception,8.8
2,Django Unchained,8.4
3,Jurassic Park,8.1


In [10]:
# Find the Top 3 Directors with the Highest Average Movie Ratings
query = '''
SELECT Directors.name, AVG(Ratings.user_rating) AS avg_rating
FROM Movies
JOIN Directors ON Movies.director_id = Directors.director_id
JOIN Ratings ON Movies.movie_id = Ratings.movie_id
GROUP BY Directors.name
ORDER BY avg_rating DESC
LIMIT 3;
'''

# Execute the query and display the results
result = pd.read_sql_query(query, connection)
result


,name,avg_rating
0,Christopher Nolan,8.80
1,Quentin Tarantino,8.65
2,Steven Spielberg,8.10


In [11]:
connection.close()
